In [ ]:
import argparse
import cv2
import numpy as np
from easydict import EasyDict
from pdb import set_trace as b

In [ ]:
parser = argparse.ArgumentParser(description="Fisheye Camera Calibration")
parser.add_argument('-id','--CAMERA_ID', default=1, type=int, help='Camera ID')
parser.add_argument('-fw','--FRAME_WIDTH', default=1280, type=int, help='Camera Frame Width')
parser.add_argument('-fh','--FRAME_HEIGHT', default=720, type=int, help='Camera Frame Height')
parser.add_argument('-bw','--N_CHESS_BORAD_WIDTH', default=6, type=int, help='Chess Board Width (corners number)')
parser.add_argument('-bh','--N_CHESS_BORAD_HEIGHT', default=8, type=int, help='Chess Board Height (corners number)')
parser.add_argument('-square','--SQUARE_SIZE_MM', default=20, type=int, help='Chess Board Square Size (mm)')
parser.add_argument('-calibrate','--N_CALIBRATE_SIZE', default=10, type=int, help='Required Calibration Frame Number')
parser.add_argument('-delay','--FIND_CHESSBOARD_DELAY_MOD', default=4, type=int, help='Find Chessboard Delay (frame number)')
parser.add_argument('-read','--MAX_READ_FAIL_CTR', default=10, type=int, help='Max Read Image Failed Criteria (frame number)')
cfgs = parser.parse_args()

In [ ]:
# cfgs = EasyDict()
# cfgs.CAMERA_ID = 1                      # 相机编号
# cfgs.FRAME_WIDTH = 1280                 # 相机分辨率 帧宽度
# cfgs.FRAME_HEIGHT = 720                 # 相机分辨率 帧高度
# cfgs.N_CHESS_BORAD_WIDTH = 6            # 棋盘宽度 【内角点数】
# cfgs.N_CHESS_BORAD_HEIGHT = 8           # 棋盘高度 【内角点数】
# cfgs.SQUARE_SIZE_MM = 20                # 棋盘格边长 mm
# cfgs.N_CALIBRATE_SIZE = 10              # 最小标定采样数量
# cfgs.FIND_CHESSBOARD_DELAY_MOD = 4      # 延时的帧数
# cfgs.FOCAL_SCALE = 1.0
# cfgs.MAX_READ_FAIL_CTR = 10             # 读取图像超时设置
cfgs.CHESS_BOARD_SIZE = lambda: (cfgs.N_CHESS_BORAD_WIDTH, cfgs.N_CHESS_BORAD_HEIGHT)   # 棋盘格尺寸

In [ ]:
flags = EasyDict()                      # 标志清零
flags.READ_FAIL_CTR = 0                 
flags.frame_id = 0
flags.ok = False

In [ ]:
BOARD = np.array([ [(j * cfgs.SQUARE_SIZE_MM, i * cfgs.SQUARE_SIZE_MM, 0.)]
    for i in range(cfgs.N_CHESS_BORAD_HEIGHT) for j in range(cfgs.N_CHESS_BORAD_WIDTH) ])

In [ ]:
class calib_t(EasyDict):
    def __init__(self):
        super().__init__({
        "type":None,                   # 自定义数据类型
        "camera_mat":None,             # 相机内参
        "dist_coeff":None,             # 畸变参数
        "rvecs":None,                  # 旋转向量
        "tvecs":None,                  # 平移向量
        "map1":None,                   # 映射矩阵1
        "map2":None,                   # 映射矩阵2
        "reproj_err":None,             # 重投影误差
        "ok":False,                    # 数据采集完成标志
        })

In [ ]:
# cv2.fisheye.calibrate ( objectPoints,    # 角点在棋盘中的空间坐标向量
#                         imagePoints,     # 角点在图像中的坐标向量
#                         image_size,      # 图片大小
#                         K,               # 相机内参矩阵
#                         D,               # 畸变参数向量
#                         rvecs,           # 旋转向量
#                         tvecs,           # 平移向量
#                         flags,           # 操作标志
#                         criteria         # 迭代优化算法的停止标准
#                         )

# flags:
#     cv2.fisheye.CALIB_USE_INTRINSIC_GUESS     # 相机内参矩阵包含有效的fx，fy，cx，cy初始值，这些值会进一步进行优化
#                                               # 否则，（cx，cy）初始化设置为图像中心（使用imageSize），并且以最小二乘法计算焦距。
#     cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC     # 在每次内部参数优化迭代之后，将重新计算外部参数。
#     cv2.fisheye.CALIB_CHECK_COND              # 检查条件编号的有效性
#     cv2.fisheye.CALIB_FIX_SKEW                # 偏斜系数（alpha）设置为零，并保持为零
#     cv2.fisheye.CALIB_FIX_K1 (K1-K4)          # 选定的畸变系数设置为零，并保持为零  CALIB_FIX_INTRINSIC则全为零

# criteria:
#     TermCriteria (int type, int maxCount, double epsilon)      # 类型、最大计数次数、最小精度
#     Criteria type, can be one of: COUNT, EPS or COUNT + EPS    # 角点优化最大迭代次数或角点优化位置移动量小于epsilon值

In [ ]:
# cv2.checkRange 检查元素非空及无异常值

In [ ]:
class Fisheye:
    def __init__(self):
        self.data = calib_t()
        self.inited = False
        
    def update(self, corners, frame_size):
        board = [BOARD] * len(corners)
        if not self.inited:
            self._update_init(board, corners, frame_size)
            self.inited = True
        else:
            self._update_refine(board, corners, frame_size)
        #self._calc_reproj_err(corners)
        
    def _update_init(self, board, corners, frame_size):
        data = self.data
        data.type = "FISHEYE"
        data.camera_mat = np.eye(3, 3)
        data.dist_coeff = np.zeros((4, 1))
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC,
            criteria=(cv2.TERM_CRITERIA_COUNT, 30, 0.1))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _update_refine(self, board, corners, frame_size):
        data = self.data
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC|cv2.CALIB_USE_INTRINSIC_GUESS,
            criteria=(cv2.TERM_CRITERIA_COUNT, 10, 0.1))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _calc_reproj_err(self, corners):
        if not self.inited: return
        data = self.data
        data.reproj_err = []
        for i in range(len(corners)):
            corners_reproj = cv2.fisheye.projectPoints(BOARD[i], data.rvecs[i], data.tvecs[i], data.camera_mat, data.dist_coeff)
            err = cv2.norm(corners_reproj, corners[i], cv2.NORM_L2)
            data.reproj_err.append(err)

In [ ]:
# cv2.findChessboardCorners ( image,              # 棋盘图像
#                             patternSize,        # 棋盘格行和列的【内角点】数量
#                             corners,            # 输出数组
#                             flags               # 操作标志
#                             )
# flags:
#     CV_CALIB_CB_ADAPTIVE_THRESH      # 使用自适应阈值处理将图像转换为黑白图像
#     CV_CALIB_CB_NORMALIZE_IMAGE      # 对图像进行归一化。
#     CV_CALIB_CB_FILTER_QUADS         # 过滤在轮廓检索阶段提取的假四边形。
#     CALIB_CB_FAST_CHECK              # 对查找棋盘角的图像进行快速检查

In [ ]:
# cv2.cornerSubPix (image,             # 棋盘图像
#                   corners,           # 棋盘角点
#                   winSize,           # 搜索窗口边长的一半
#                   zeroZone,          # 搜索区域死区大小的一半, (-1,-1)代表无
#                   criteria           # 迭代停止标准
#                  )

In [ ]:
class data_t(EasyDict):                # 棋盘寻找和亚像素优化 得到角点坐标
    def __init__(self, raw_frame):
        super().__init__({
        "raw_frame":raw_frame,
        "corners":None,
        "ok":False,
        })
        # 寻找棋盘角点
        self.ok, self.corners = cv2.findChessboardCorners(self.raw_frame, cfgs.CHESS_BOARD_SIZE(),
                                flags = cv2.CALIB_CB_ADAPTIVE_THRESH|cv2.CALIB_CB_NORMALIZE_IMAGE|cv2.CALIB_CB_FAST_CHECK)
        if not self.ok: return
        # 角点坐标亚像素优化
        gray = cv2.cvtColor(self.raw_frame, cv2.COLOR_BGR2GRAY)
        self.corners = cv2.cornerSubPix(gray, self.corners, (11, 11), (-1, -1),
                        (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1))

In [ ]:
class history_t:                       # 定义历史存储数据类及相应操作
    def __init__(self):
        self.corners = []
        self.updated = False
    def append(self, current):
        if not current.ok: return
        self.corners.append(current.corners)
        self.updated = True
    def removei(self, i):
        if not 0 <= i < len(self): return
        del self.corners[i]
        self.updated = True
    def __len__(self):
        return len(self.corners)
    def get_corners(self):
        self.updated = False
        return self.corners

In [ ]:
# cv2.fisheye.initUndistortRectifyMap (K,         # 相机内参矩阵
#                                      D,         # 畸变向量
#                                      R,         # 旋转矩阵
#                                      P,         # 新的相机矩阵
#                                      size,      # 输出图像大小
#                                      m1type,    # 映射矩阵类型
#                                      map1,      # 输出映射矩阵1
#                                      map2       # 输出映射矩阵2
#                                     )

In [ ]:
def main():
    history = history_t()

    cap = cv2.VideoCapture(cfgs.CAMERA_ID)                                   # 开启相机
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cfgs.FRAME_WIDTH)                      # 设置分辨率
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cfgs.FRAME_HEIGHT)
    if not cap.isOpened(): 
        raise Exception("camera {} open failed".format(cfgs.CAMERA_ID))      # 相机开启失败

    fisheye = Fisheye()                                                      # 鱼眼相机类

    while True:
        ok, raw_frame = cap.read()                                           # 从相机读入原始帧
        if not ok:
            flags.READ_FAIL_CTR += 1
            if flags.READ_FAIL_CTR >= cfgs.MAX_READ_FAIL_CTR:                # 读取图像超时
                raise Exception("image read failed")
        else:
            flags.READ_FAIL_CTR = 0
            flags.frame_id += 1

        if 0 == flags.frame_id % cfgs.FIND_CHESSBOARD_DELAY_MOD:             # 间隔一定帧数采样
            current = data_t(raw_frame)                                      # 得到棋盘角点坐标
            history.append(current)                                          # 存储数据

        if len(history) >= cfgs.N_CALIBRATE_SIZE and history.updated:        # 达到标定采样数量且数据更新时开始标定
            fisheye.update(history.get_corners(), raw_frame.shape[1::-1])    # 更新标定参数
            calib = fisheye.data                                             # 得到标定参数 K D                     
            calib.map1, calib.map2 = cv2.fisheye.initUndistortRectifyMap(
                calib.camera_mat, calib.dist_coeff, np.eye(3, 3), calib.camera_mat, raw_frame.shape[1::-1], cv2.CV_16SC2)
                                                                             # 得到去畸变映射map矩阵
        if len(history) >= cfgs.N_CALIBRATE_SIZE:
            undist_frame = cv2.remap(raw_frame, calib.map1, calib.map2, cv2.INTER_LINEAR);     
            cv2.imshow("undist_frame", undist_frame)                         # 重映射输出 去畸变图像

        cv2.imshow("raw_frame", raw_frame)
        key = cv2.waitKey(1)
        if key == 27: break                                                  # esc退出
    
    cv2.destroyAllWindows() 
    
    if fisheye.data.dist_coeff is None:
        raise Exception("calibration failed. chessboard not found, check the parameters")  # 标定失败 未找到棋盘或参数设置错误
    print("Calibration Complete")
    print("Camera Matrix is : {}".format(fisheye.data.camera_mat.tolist()))
    print("Distortion Coefficient is : {}".format(fisheye.data.dist_coeff.tolist()))
    np.save('camera_K.npy',fisheye.data.camera_mat.tolist())
    np.save('camera_D.npy',fisheye.data.dist_coeff.tolist())                 # 输出并存储数据
    
if __name__ == '__main__':
    main()
